Task 6

a)

Hash with xxhash and a seed

In [12]:
import xxhash

def hash_with_xxhash(vector, seed):
    h = xxhash.xxh32(seed=seed)
    h.update(str(vector.tolist()))
    return h.intdigest()

Hash with built-in method

In [13]:
def hash_with_builtin(vector, band_index):
    """Hash a vector using Python's built-in hash function."""
    combined = str(vector.tolist()) + f"_band_{band_index}"
    return hash(combined)

In [14]:
import numpy as np

def lsh_hash_functions(signature_matrix, b, r, method="xxhash"):
    """
    Constructs hash functions for LSH.
    
    Args:
        signature_matrix (np.ndarray): Signature matrix (rows x columns).
        b (int): Number of bands.
        r (int): Number of rows in each band.
        method (str): Hash method, either "xxhash" or "builtin".
    
    Returns:
        list: List of bucket indices for each column in the matrix.
    """
    n_columns = signature_matrix.shape[1]
    assert signature_matrix.shape[0] == b * r, "Matrix rows must equal b * r"

    buckets = []
    for band_index in range(b):
        start_row = band_index * r
        end_row = start_row + r
        band = signature_matrix[start_row:end_row, :] 
        
        band_hashes = []
        for col in range(n_columns):
            vector = band[:, col]
            if method == "xxhash":
                seed = band_index + 1 
                bucket = hash_with_xxhash(vector, seed)
            elif method == "builtin":
                bucket = hash_with_builtin(vector, band_index)
            else:
                raise ValueError("Invalid method. Use 'xxhash' or 'builtin'.")
            
            band_hashes.append(bucket)
        buckets.append(band_hashes)
    
    return buckets


Run the code

In [15]:
b = 4 
r = 2 
s = 0.8 
N = 10 

K = b * r
n_columns = 5
signature_matrix = np.random.randint(0, N, (K, n_columns))

xxhash_buckets = lsh_hash_functions(signature_matrix, b, r, method="xxhash")
builtin_buckets = lsh_hash_functions(signature_matrix, b, r, method="builtin")

print("Signature Matrix:\n", signature_matrix)
print("\nBuckets using xxhash:\n", xxhash_buckets)
print("\nBuckets using Python's built-in hash:\n", builtin_buckets)

Signature Matrix:
 [[5 5 0 7 5]
 [2 8 1 7 9]
 [2 4 5 9 5]
 [3 2 3 0 3]
 [0 0 9 5 4]
 [3 2 0 5 1]
 [7 9 4 6 9]
 [1 7 1 3 0]]

Buckets using xxhash:
 [[1824749699, 4020557567, 4079688665, 1803803446, 1286474314], [1850315393, 391800488, 3859264596, 25947172, 3859264596], [3737323855, 3376518533, 3347375252, 1409440022, 1439098364], [867847304, 1322268635, 3964253424, 2255015997, 2474378130]]

Buckets using Python's built-in hash:
 [[2694800603913483724, -8553850955125771678, 4117684228793142541, 1048294759488727680, -7909327128676437556], [-986946687483998108, 6532627136569189228, 4792866662828641366, 6087755270484007340, 4792866662828641366], [-803155391544906301, 3732671219900517412, 3096101499897109505, -3994460969145757436, -3804314465236136739], [-5063472191170282312, -184467117567400726, 8711283641133739627, -8365980334279249814, 1717304300677024609]]


b)

In [16]:
from collections import defaultdict

def lsh(signature_matrix, b, r, method="xxhash"):
    """
    Implements Locality Sensitive Hashing (LSH).
    
    Args:
        signature_matrix (np.ndarray): Signature matrix (rows x columns).
        b (int): Number of bands.
        r (int): Number of rows in each band.
        method (str): Hash method, either "xxhash" or "builtin".
    
    Returns:
        list of defaultdict: A list where each entry represents a band and maps bucket indices
                             to the list of column indices.
    """
    n_columns = signature_matrix.shape[1]
    assert signature_matrix.shape[0] == b * r, "Matrix rows must equal b * r"

    hash_tables = []

    for band_index in range(b):
        start_row = band_index * r
        end_row = start_row + r
        band = signature_matrix[start_row:end_row, :]
        
        band_buckets = defaultdict(list)

        for col_index in range(n_columns):
            vector = band[:, col_index] 
            
            if method == "xxhash":
                seed = band_index + 1 
                bucket = hash_with_xxhash(vector, seed)
            elif method == "builtin":
                bucket = hash_with_builtin(vector, band_index)
            else:
                raise ValueError("Invalid method. Use 'xxhash' or 'builtin'.")
            
            if bucket > 2**31 - 1:
                bucket = str(bucket)
            
            band_buckets[bucket].append(col_index)
        
        # Store the hash table for the band
        hash_tables.append(band_buckets)
    
    return hash_tables

Run the code:

In [17]:
b = 4 
r = 2  
s = 0.8 
N = 10 

K = b * r
n_columns = 5
signature_matrix = np.random.randint(0, N, (K, n_columns))

hash_tables = lsh(signature_matrix, b, r, method="xxhash")

print("Signature Matrix:\n", signature_matrix)
for band_index, hash_table in enumerate(hash_tables):
    print(f"\nBand {band_index + 1}:")
    for bucket, columns in hash_table.items():
        print(f"  Bucket {bucket}: Columns {columns}")

Signature Matrix:
 [[4 8 0 8 7]
 [5 6 2 0 4]
 [1 4 7 8 1]
 [2 2 0 7 5]
 [7 8 4 2 4]
 [4 4 2 9 3]
 [8 9 8 1 7]
 [7 6 7 2 5]]

Band 1:
  Bucket 2786887984: Columns [0]
  Bucket 2839974396: Columns [1]
  Bucket 1127391875: Columns [2]
  Bucket 3233395022: Columns [3]
  Bucket 1119608697: Columns [4]

Band 2:
  Bucket 269785389: Columns [0]
  Bucket 391800488: Columns [1]
  Bucket 272369360: Columns [2]
  Bucket 3389864645: Columns [3]
  Bucket 598234658: Columns [4]

Band 3:
  Bucket 4142203178: Columns [0]
  Bucket 3738571090: Columns [1]
  Bucket 1371029930: Columns [2]
  Bucket 1896144446: Columns [3]
  Bucket 2242143040: Columns [4]

Band 4:
  Bucket 2929913968: Columns [0, 2]
  Bucket 3611911972: Columns [1]
  Bucket 2688469960: Columns [3]
  Bucket 2805241080: Columns [4]


c)

In [18]:
def report_lsh_buckets(signature_matrix, b, r, method="xxhash"):
    """
    Analyzes the MinHash signature using LSH and identifies buckets with collisions.
    
    Args:
        signature_matrix (np.ndarray): MinHash signature matrix.
        b (int): Number of bands.
        r (int): Number of rows per band.
        method (str): Hashing method ("xxhash" or "builtin").
    
    Returns:
        dict: Dictionary containing bucket information for each band.
              Format: {band_index: [(bucket, bucket_size, column_ids)]}
    """
    n_columns = signature_matrix.shape[1]
    assert signature_matrix.shape[0] == b * r, "Matrix rows must equal b * r"

    hash_tables = lsh(signature_matrix, b, r, method=method)

    results = {}
    for band_index, hash_table in enumerate(hash_tables):
        band_results = []
        for bucket, columns in hash_table.items():
            if len(columns) > 1:  # At least two columns in the bucket
                band_results.append((bucket, len(columns), columns))
        if band_results:
            results[band_index] = band_results
    return results

Run Code

In [19]:
b = 20 
r = 5 
s = 0.8 
N = 10**8 

K = b * r
n_columns = 10 
np.random.seed(42)
signature_matrix = np.random.randint(0, N, (K, n_columns))

results = report_lsh_buckets(signature_matrix, b, r, method="xxhash")
print("Signature Matrix:\n", signature_matrix)
for band_index, collisions in results.items():
    print(f"\nBand {band_index + 1}:")
    for bucket, bucket_size, columns in collisions:
        print(f"  Bucket {bucket}: Size {bucket_size}, Columns {columns}")

Signature Matrix:
 [[65682867 56755036 56882282 21081788 13315092 35788921 26735830 93410762
  96319575 91090292]
 [31632483 76737383 88358551 88409749  4981505 13953367 85652971  4521373
   3344769 98750923]
 [76893497 30349564 99052376 42860080 77751354 62250665 66209791 46792155
  21498555 97117135]
 [60221198 79757501 16861870 52286002 32049003 61136438 62194931 65285250
  69537252 59248434]
 [61306900 55831368 10959014 56972561 48900483 20193880  3385357 44738553
  68574553 16845364]
 [52157313 95672411 11392366 69778859 88883975 13479854 73506850 13187277
  68979792 37709731]
 [26939239 23027075 81200125 86191493 87796277 66401385 22335235   271836
   3584702 52631083]
 [ 8585377 86544585 85157821 87655395 17824013 36601694 67105583 60304654
  34119117 36433622]
 [89482491 60031992 55105831  3366612 58743503 91693393 36258670 69739572
  55848857 57055419]
 [49160571 68394024 64525468 89966606 38727468 51757120 67157848 77897542
  73665032 95540567]
 [85390464 48170987 52562567 23

d)

Jaccard Similarity method

In [20]:
def calculate_jaccard_similarity(sets, col1, col2):
    """Calculate Jaccard similarity between two columns."""
    set1, set2 = sets[col1], sets[col2]
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union != 0 else 0

Signature Similarity Method

In [21]:
def calculate_signature_similarity(signature_matrix, col1, col2):
    """Calculate signature similarity between two columns."""
    sig1, sig2 = signature_matrix[:, col1], signature_matrix[:, col2]
    matches = np.sum(sig1 == sig2)
    return matches / len(sig1)

Method to analyze the groups

In [29]:
from itertools import combinations

def analyze_largest_group(signature_matrix, sets, group_columns):
    """
    Analyze the largest group of columns by comparing Jaccard and signature similarities.
    
    Args:
        signature_matrix (np.ndarray): MinHash signature matrix.
        sets (list of sets): Original sets corresponding to the columns.
        group_columns (list): Indices of the columns in the group.
    
    Returns:
        list: Results for each pair in the group.
    """

    if not group_columns:
        print("Debug: group_columns is empty!")
        return []

    for col in group_columns:
        if col >= signature_matrix.shape[1]:
            raise ValueError(f"Column index {col} is out of bounds for signature matrix.")
        if col >= len(sets):
            raise ValueError(f"Column index {col} is out of bounds for sets.")

    if len(group_columns) < 2:
        print("Debug: group_columns has fewer than 2 elements.")
        return []

    results = []
    for col1, col2 in combinations(group_columns, 2):
        print(f"Debug: Comparing columns {col1} and {col2}")
        jaccard_sim = calculate_jaccard_similarity(sets, col1, col2)
        signature_sim = calculate_signature_similarity(signature_matrix, col1, col2)
        results.append((col1, col2, jaccard_sim, signature_sim))
    return results


Run code

In [32]:
sets = [
    {1, 2, 3}, {2, 3, 4}, {4, 5, 6}, {1, 2}, {2, 3}, {1, 3, 5}, {4, 5}, {6, 7}, {2, 4}, {3, 6}
]
b = 5
r = 4
n_columns = len(sets)
K = b * r
N = 10**6 

np.random.seed(42)
signature_matrix = np.random.randint(0, N, (K, n_columns))

results = {
    0: [(1234, 4, [0, 1, 2, 3])],
    1: [(5678, 3, [4, 5, 6])],
}

filtered_groups = [
    group for band in results.values() for group in band if len(group[2]) <= 10
]
largest_group = max(filtered_groups, key=lambda x: len(x[2]))
group_columns = largest_group[2]

similarity_results = analyze_largest_group(signature_matrix, sets, group_columns)

print(f"Largest Group: {group_columns}")
for col1, col2, jaccard_sim, signature_sim in similarity_results:
    print(f"Columns {col1}-{col2}: Jaccard = {jaccard_sim:.3f}, Signature = {signature_sim:.3f}")

Debug: Comparing columns 0 and 1
Debug: Comparing columns 0 and 2
Debug: Comparing columns 0 and 3
Debug: Comparing columns 1 and 2
Debug: Comparing columns 1 and 3
Debug: Comparing columns 2 and 3
Largest Group: [0, 1, 2, 3]
Columns 0-1: Jaccard = 0.500, Signature = 0.000
Columns 0-2: Jaccard = 0.000, Signature = 0.000
Columns 0-3: Jaccard = 0.667, Signature = 0.000
Columns 1-2: Jaccard = 0.200, Signature = 0.000
Columns 1-3: Jaccard = 0.250, Signature = 0.000
Columns 2-3: Jaccard = 0.000, Signature = 0.000
